In [11]:
def sigmoid(x):
    return 1 / (1 + pow(2.71828, -x))

class CustomRandom:
    def __init__(self, seed=123456789):
        self.seed = seed
        self.mult = 16807
        self.mod = (2**31) - 1
        self.current = seed
    
    def random(self):
        self.current = (self.mult * self.current + 1) % self.mod
        return self.current / self.mod # If it [0,10) then multiply by 10
    
    def random_list(self, size):
        return [self.random() for _ in range(size)]

custom_random = CustomRandom()

class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        self.output = 0

    def activate(self, inputs):
        total = sum(w * i for w, i in zip(self.weights, inputs)) + self.bias
        self.output = sigmoid(total)
        return self.output

class Layer:
    def __init__(self, num_neurons, num_inputs, weight_data=None):
        self.neurons = []
        for i in range(num_neurons):
            if weight_data:
                weights = weight_data.pop(0)
                bias = float(weight_data.pop(0)[0])
            else:
                weights = [custom_random.random() for _ in range(num_inputs)]
                bias = custom_random.random()
            self.neurons.append(Neuron(weights, bias))

    def forward(self, inputs):
        return [neuron.activate(inputs) for neuron in self.neurons]

class NeuralNetwork:
    def __init__(self, structure_file, weight_file=None):
        with open(structure_file) as f:
            lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]
            self.num_layers = int(lines[0])
            self.layer_sizes = list(map(int, lines[1:self.num_layers+1]))

        if weight_file:
            with open(weight_file) as f:
                raw = [line.strip() for line in f if line.strip() and not line.startswith("#")]
                weight_data = []
                i = 0
                while i < len(raw):
                    weight_row = list(map(float, raw[i].split()))
                    if len(weight_row) == 1:
                        weight_data.append(weight_row)
                    else:
                        weight_data.append(weight_row)
                    i += 1
        else:
            weight_data = None

        self.layers = []
        for i in range(1, len(self.layer_sizes)):
            inputs = self.layer_sizes[i - 1]
            neurons = self.layer_sizes[i]
            if weight_data:
                neurons_data = []
                for _ in range(neurons):
                    weights = weight_data.pop(0)
                    bias = weight_data.pop(0)
                    neurons_data.append((weights, bias))
                layer = Layer(neurons, inputs, [[*w] for w, b in neurons_data for w in [w]] + [b for w, b in neurons_data])
            else:
                layer = Layer(neurons, inputs)
            self.layers.append(layer)

    def feedforward(self, input_data):
        for layer in self.layers:
            input_data = layer.forward(input_data)
        return input_data

if __name__ == "__main__":
    print("XOR Network:")
    nn = NeuralNetwork("network.txt", "weights.txt")  

    test_cases = [
        ([0, 0], 0),
        ([0, 1], 1),
        ([1, 0], 1),
        ([1, 1], 0)
    ]

    for inputs, expected in test_cases:
        output = nn.feedforward(inputs)[0]
        print(f"Input: {inputs}, Output: {round(output, 3)}, Expected: {expected}")

XOR Network:
Input: [0, 0], Output: 0.872, Expected: 0
Input: [0, 1], Output: 0.871, Expected: 1
Input: [1, 0], Output: 0.894, Expected: 1
Input: [1, 1], Output: 0.872, Expected: 0
